In [1]:
import tkinter as tk
from tkinter import *
import tkinter.messagebox as tkmb
import PIL
from PIL import Image, ImageTk
import cv2 #for visualization
import numpy as np
import threading
import time
from threading import Thread
import cv2
from tkinter import filedialog
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

class ObjectDetection(object):
    def __init__(self, num):
        self.capture = cv2.VideoCapture(0) #opens laptops video cam if you want cctv add ip address(192)
        
        self.yolo = cv2.dnn.readNet("./yolov3.cfg","./yolov3.weights")

        self.classes=[] #80 classes
        with open("./coco.names" ,"r")as f:
            self.classes=f.read().splitlines()
        
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

    def update(self):
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
    def show_frame(self):
        try:
            
            if self.status:
                img = cv2.cvtColor(self.frame,cv2.COLOR_BGR2RGB)
                blob = cv2.dnn.blobFromImage(self.frame,1/255.0,(416,416),(0,0,0),swapRB=True,crop=False) #convert image into grids
                self.yolo.setInput(blob) #sends input to yolo variable

                output_layer_names= self.yolo.getUnconnectedOutLayersNames()#gets class names of objects in the given image
                layeroutput = self.yolo.forward(output_layer_names) #output

                boxes =[]
                confidences=[]
                class_ids=[]
                h, w = img.shape[:2] #gives height and width of image
                for output in layeroutput:
                    for detection in output:
                        scores = detection[5:] #list of 80 items ie.classes with respect to accuracy so if bicyle is present only one index wil have accuracy(0.94) others will be given 0 
                        classID = np.argmax(scores)

                        confidence = scores[classID] #accuracy

                        if confidence > 0.4:
                            box = detection[:4] * np.array([w, h, w, h])
                            (centerX, centerY, width, height) = box.astype("int")
                            x = int(centerX - (width / 2))
                            y = int(centerY - (height / 2))
                            box = [x, y, int(width), int(height)]
                            boxes.append(box)
                            confidences.append(float(confidence))
                            class_ids.append(classID)
                indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4) #as image is divided into grids, there will be 4 predictions for bicylce so this function will reduce these prediction to single box
                font = cv2.FONT_HERSHEY_PLAIN
                colors = np.random.uniform(0,255,size=(len(boxes),3))
                if(len(boxes)!=0):
                    for i in indexes.flatten():
                        x,y,w,h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        conf = str(round(confidences[i],2))
                        color = colors[i]
                        cv2.rectangle(img,(x,y),(x+w,y+h),color,10)
                        cv2.putText(img,label+" "+conf,(x,y+20),font,2,(255,255,255),2)
                    
                    img1 = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=img1.resize((400, 400)))
                    lmain.imgtk = imgtk
                    lmain.configure(image=imgtk)
                    lmain.after(0, self.show_frame())#this method is running infin
                else:
                    img = cv2.cvtColor(self.frame,cv2.COLOR_BGR2RGB)
                    img1 = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=img1.resize((400, 400)))
                    lmain.imgtk = imgtk
                    lmain.configure(image=imgtk)
                    lmain.after(2, self.show_frame())
                    
        except tk.TclError:
            print(" ")
            
def cam(number):
    rtsp_stream_link = number
    video_stream_widget = ObjectDetection(number)
    while True:
        try:
            video_stream_widget.show_frame()
        except AttributeError:
            pass
        

class ImageDetection(object):
    def __init__(self):
        self.yolo = cv2.dnn.readNet("./yolov3.cfg","./yolov3.weights")

        self.classes=[]
        with open("./coco.names" ,"r")as f:
            self.classes=f.read().splitlines()
        self.newWindow =tk.Toplevel(root)
        self.newWindow.title("Image Detection")

        self.newWindow.geometry("400x480")

        self.lbl = tk.Label(self.newWindow,text='waiting.........')
        self.lbl.place(x=0, y=0)
        
    def getfileImage(self):
        importfilepath = filedialog.askopenfilename() #opens our directory to take image
        img = cv2.imread(importfilepath)
        if img is None or img.size == 0:
            return
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        blob = cv2.dnn.blobFromImage(img,1/255.0,(416,416),(0,0,0),swapRB=True,crop=False)
        self.yolo.setInput(blob)
        output_layer_names=self.yolo.getUnconnectedOutLayersNames()
        layeroutput = self.yolo.forward(output_layer_names)
        boxes =[]
        confidences=[]
        class_ids=[]
        h, w = img.shape[:2]
        for output in layeroutput:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)  
                confidence = scores[classID]
                if confidence > 0.5:
                    box = detection[:4] * np.array([w, h, w, h])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    box = [x, y, int(width), int(height)]
                    boxes.append(box)
                    confidences.append(float(confidence))
                    class_ids.append(classID)
        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0,255,size=(len(boxes),3))
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label =  str(self.classes[class_ids[i]])
            conf = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,10)
            cv2.putText(img,label+" "+conf,(x,y+20),font,2,(255,255,255),2)
        img1 = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img1.resize((400, 400)))

        self.lbl.imgtk = imgtk
        self.lbl.configure(image=imgtk)
        btnSave = tk.Button(self.newWindow, text='Save Image', bd='5',width=10,bg='red',
                command=lambda : self.saveimg(img))
        btnSave.place(x=150,y=420)
    
    def saveimg(self,img):
        exportfilepath = filedialog.asksaveasfilename(defaultextension='.jpg')

        cv2.imwrite(exportfilepath,cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        tkmb.showinfo("showinfo", "Image Saved Successfully..")
        self.newWindow.destroy()
        
class VideoDetection(object):
    def __init__(self):
        self.yolo = cv2.dnn.readNet("./yolov3.cfg","./yolov3.weights")

        self.classes=[]
        with open("./coco.names" ,"r")as f:
            self.classes=f.read().splitlines()
        self.newWindow =tk.Toplevel(root)
        self.newWindow.title("Video Detection")

        self.newWindow.geometry("400x480")

        self.lbl = tk.Label(self.newWindow,text='waiting.........')
        self.lbl.place(x=0, y=0)
        importfilepath = filedialog.askopenfilename()
        self.cap = cv2.VideoCapture(importfilepath)

        
        self.lbl1 = tk.Label(self.newWindow,text='Click Run Video ')
        self.lbl1.place(x=10, y=2)
        self.lbl = tk.Label(self.newWindow)
        self.lbl.place(x=0, y=0)
        self.btnRun = tk.Button(self.newWindow, text='Run Video', bd='5',width=10,bg='red',
                    command=self.run_video)
        self.btnRun.place(x=120,y=420)
        self.btnSave = tk.Button(self.newWindow, text='Save Video', bd='5',width=10,bg='red',
                    command=self.save_video)
        self.btnSave.place(x=220,y=420)
        self.exportfilepath=''
        self.out=''

    def run_video(self):
        self.exportfilepath = filedialog.asksaveasfilename(defaultextension='.avi')
        codec = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
        frame_width = int(self.cap.get(3))
        frame_height = int(self.cap.get(4))
        self.out = cv2.VideoWriter(self.exportfilepath, codec, 30.0, (frame_width, frame_height))

        t = threading.Thread(target=self.stream)
        t.start()
    
    def stream(self):
        while True:
            status,img =self.cap.read()
            if img is None or img.size == 0:
                return
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            blob = cv2.dnn.blobFromImage(img,1/255.0,(416,416),(0,0,0),swapRB=True,crop=False)

            self.yolo.setInput(blob)
            output_layer_names=self.yolo.getUnconnectedOutLayersNames()
            layeroutput = self.yolo.forward(output_layer_names)
            boxes =[]
            confidences=[]
            class_ids=[]
            h, w = img.shape[:2]
            for output in layeroutput:
                for detection in output:
                    scores = detection[5:]
                    classID = np.argmax(scores)  
                    confidence = scores[classID]
                    if confidence > 0.5:
                        box = detection[:4] * np.array([w, h, w, h])
                        (centerX, centerY, width, height) = box.astype("int")
                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))
                        box = [x, y, int(width), int(height)]
                        boxes.append(box)
                        confidences.append(float(confidence))
                        class_ids.append(classID)
            indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
            font = cv2.FONT_HERSHEY_PLAIN
            colors = np.random.uniform(0,255,size=(len(boxes),3))
            if(len(indexes)>0):
                for i in indexes.flatten():
                    x,y,w,h = boxes[i]
                    label =  str(self.classes[class_ids[i]])
                    conf = str(round(confidences[i],2))
                    color = colors[i]
                    cv2.rectangle(img,(x,y),(x+w,y+h),color,10)
                    cv2.putText(img,label+" "+conf,(x,y+20),font,2,(255,255,255),2)
            img1 = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img1.resize((400, 400)))
            self.out.write(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
            self.lbl.imgtk = imgtk
            self.lbl.config(image=imgtk)
    

        

    def save_video(self):
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()
        tkmb.showinfo("showinfo", "Video Saved Successfully..")
        self.newWindow.destroy()

        
def getwebcam():
    global newWindow,lmain
    newWindow =tk.Toplevel(root)
    newWindow.title("Live Detection")

    newWindow.geometry("400x480")
    lmain = tk.Label(newWindow,text='Starting....................')
    lmain.place(x=0, y=0)
    t1 = threading.Thread(target=cam, args=(0,))
    t1.start()
    
def image_detection():
    image = ImageDetection()
    image.getfileImage()
    
def video_detection():
    image = VideoDetection()
    
if __name__ == '__main__':
    root = tk.Tk()
    root.geometry("750x394")
    root.title("Object Detection")
    root.configure(bg="black")
    
    width = 750
    height = 394
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width/2) - (width/2)
    y = (screen_height/2) - (height/2)
    root.geometry("%dx%d+%d+%d" % (width, height, x, y))
    
    # create a label widget for the background image
    bg_image = tk.PhotoImage(file="./tkBackground.png")
    bg_label = tk.Label(root, image=bg_image)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
    
    # create the buttons
    button1 = tk.Button(root, text="Add Image", font=("Helvetica", 16), command=image_detection, bd="5")
    button1.place(x=320,y=170)

    button2 = tk.Button(root, text="Add Video", font=("Helvetica", 16), command=video_detection, bd="5")
    button2.place(x=320,y=230)

    button3 = tk.Button(root, text="Live Detection", font=("Helvetica", 16), command=getwebcam, bd="5")
    button3.place(x=300,y=290)
 

    root.mainloop()
    